# 책 추천 시스템
책 제목을 넣으면 유사한 책을 리스팅해서 보여줌

In [ ]:
!curl -0 https://drive.google.com/file/d/15Q7DZ7xrJsI2Hji-WbkU9j1mwnODBd5A/view?usp=sharing

In [ ]:
import pandas as pd

In [ ]:
df_data = pd.read_csv('./0825data.csv')
df_data.head(4)

,Unnamed: 0,Desc,Unnamed: 0.1,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing


In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    2382 non-null   int64  
 1   Desc          2382 non-null   object 
 2   Unnamed: 0.1  1185 non-null   float64
 3   author        2382 non-null   object 
 4   genre         2382 non-null   object 
 5   image_link    2382 non-null   object 
 6   rating        2382 non-null   float64
 7   title         2382 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 149.0+ KB


In [ ]:
df_data['Desc'] # 활용할 컬럼 선택

0       We know that power is shifting: From West to E...
1       Following the success of The Accidental Billio...
2       How to tap the power of social software and ne...
3       William J. Bernstein is an American financial ...
4       Amazing book. And I joined Steve Jobs and many...
                              ...                        
2377    Ralph Roberts, a sus setenta años y tras la mu...
2378    Murder at the Vicarage marks the debut of Agat...
2379    In 1951 John Wyndham published his novel The D...
2380    This now classic book revealed Flannery O'Conn...
2381    Imbued on every page with Frank McCourt's asto...
Name: Desc, Length: 2382, dtype: object

# nltk tokenize를 활용하여 단어 만들기

spacebar로 나뉜 단어조합의 문자를 token한 뒤 문장으로 연결.
apply()에 넣기 전에 함수를 먼저 만든다.

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
text01 = ' We know that power is shifting: From West to E'
 
def remove_punctuation(text):
  tokenizer = RegexpTokenizer('[a-zA-Z]+')  
  text_list = tokenizer.tokenize(text)
  # print(type(text_list),text_list)
  result = ' '.join(text_list) ## string으로 연결된 값이 나옴
  # print(type(result),result)
  return result

In [ ]:
remove_punctuation(text01)

'We know that power is shifting From West to E'

In [ ]:
# Tokenizing을 위한 apply 함수 적용
# line by line
df_data['cleaned'] = df_data['Desc'].apply(remove_punctuation)

In [ ]:
df_data.columns

Index(['Unnamed: 0', 'Desc', 'Unnamed: 0.1', 'author', 'genre', 'image_link',
       'rating', 'title', 'cleaned'],
      dtype='object')

In [ ]:
df_data['cleaned'].head(5)

0    We know that power is shifting From West to Ea...
1    Following the success of The Accidental Billio...
2    How to tap the power of social software and ne...
3    William J Bernstein is an American financial t...
4    Amazing book And I joined Steve Jobs and many ...
Name: cleaned, dtype: object

# Pre-Trained

링크를 통해 colab 실행파일 내에 .csv파일을 불러오는 코드

In [ ]:
!curl -0 https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

In [ ]:
!curl -O https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1570M  100 1570M    0     0  43.5M      0  0:00:36  0:00:36 --:--:-- 44.0M


Word2Vec에 필요한 라이브러리 설치

In [ ]:
from gensim.models import Word2Vec

연관성에 의해 새로 자리잡은 단어들. 어떤 데이터를 넣으면, 매칭이 될 뿐이지 새로 자리잡는게 아님. 그 값을 유지시키는 원리<br>
free-trade 연관성 높은 단어 중 없는 단어도 있을 것. 내가 수집한 단어를 넣으면 알맞은 위치를 스스로 찾아간다(학습, 교육)

In [ ]:
# space로 분할된 단어를 다시 리스트화
corpus = []
for words in df_data['cleaned'] :
  corpus.append(words.split())
type(corpus), corpus[4:8]
# 출력 결과 확인
# (list,
#  [['Amazing',
#    'book',
#    'And',
#    'I',
#    'joined',
#    'Steve',
#    'Jobs',
#    'and',
#    'many',
#    'more',
#    'from',
#    'the',
#    'Akio',

In [ ]:
word2vec_model = Word2Vec(min_count=2, workers=-1, size=300)
word2vec_model.build_vocab(corpus)

In [ ]:
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)

In [ ]:
word2vec_model.train(corpus, total_examples=word2vec_model.corpus_count, epochs=15)

(0, 0)

In [ ]:
# 차원에 의한 벡터라이즈 값이 들어있다.
word2vec_model.wv.vocab

## 출력 결과
# {'We': <gensim.models.keyedvectors.Vocab at 0x7f31c2326d10>,
#  'know': <gensim.models.keyedvectors.Vocab at 0x7f31c2333090>,
#  'that': <gensim.models.keyedvectors.Vocab at 0x7f31c2333f90>,
#  'power': <gensim.models.keyedvectors.Vocab at 0x7f31c2329d10>,
#  'is': <gensim.models.keyedvectors.Vocab at 0x7f31c2329f50>,
#  'shifting': <gensim.models.keyedvectors.Vocab at 0x7f31c2329c50>,
#  'From': <gensim.models.keyedvectors.Vocab at 0x7f31c2329150>,
#  'West': <gensim.models.keyedvectors.Vocab at 0x7f31c3e2cdd0>,
#  'to': <gensim.models.keyedvectors.Vocab at 0x7f31c2329e50>,
#  'East': <gensim.models.keyedvectors.Vocab at 0x7f31c2329510>,

# 벡터시키는 작업

## for 문 설명
- 라인 출력
- 단어를 출력
- vocabulary와 매칭하기 :       
      # 단어마다 word2vec에 저장된 숫자가 나온다. 그냥 이런 숫자가 들어가있다. 
      # 현재 300 차원. 글자 표현도 300차원의 숫자로.
      # high는 [300개숫자] 로 된 것. 벡터로 점찍는 것. 그 문장이 어떤 벡터를 갖고 있는가가 1개의 벡터로 다시 표현되어야 하기 때문에
      # 덧셈으로 작업한다. 벡터플러스(사이즈가 똑같을 때=300차원 동일할 때)
      # print(word2vec_model[word]) 차원값 확인
- 

In [ ]:
doc_embedding_list = list()
# for line in df_data['cleaned'][2:3]:
for line in df_data['cleaned']:
  # print(line)
  doc2vec = None
  for word in line.split():
    # print(word)
    if word in word2vec_model.wv.vocab:
      # print(word2vec_model[word])
      if doc2vec is None:
        doc2vec = word2vec_model[word] 
      else :
        doc2vec = doc2vec + word2vec_model[word] 

  if doc2vec is not None:
    doc2vec = doc2vec / len(doc2vec)
    doc_embedding_list.append(doc2vec)
  # print(type(doc2vec), doc2vec.shape, doc2vec)

print(doc_embedding_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
type(doc_embedding_list), doc_embedding_list[3]

# (list,
#  array([ 4.86756396e-03,  6.42052153e-03,  2.03044293e-03,  7.35262316e-03,
#         -1.05007330e-03, -4.15433245e-03,  4.42025298e-03, -1.21672340e-02,
#          1.21027296e-02, -1.17696523e-04, -5.33394050e-03, -7.44244410e-03,
#          1.48862833e-03,  8.40701349e-03, -1.47739518e-02,  8.36980157e-03,
#          3.05416342e-03,  9.42082144e-03,  4.09478758e-04, -6.33179327e-04,
#         -1.26164081e-03,  2.53311195e-03, -2.32126680e-03, -1.08690979e-03,
#          4.72839409e-03, -7.97854271e-04, -1.40225254e-02,  8.91483948e-03,
#         -2.49086926e-03,  5.60364919e-04,  8.62206612e-03, -1.96329597e-03,
#         -5.47893066e-03, -3.32143303e-04,  8.50881450e-03,  7.87533540e-03,
#          2.39597447e-03, -6.10140734e-04,  1.08110448e-02,  4.60837735e-03,
#          8.51122197e-03,  7.39586353e-03,  4.84248856e-03,  2.98650586e-03,
#         -2.33655423e-03, -1.40437041e-03, -2.40191212e-03,  4.70298762e-03,
#          5.40223252e-03, -2.30985484e-03,  4.38869512e-03,  5.87910321e-03,
#         -3.81805818e-03, -4.21494432e-03, -1.69185922e-03,  8.39528628e-03,
#         -4.53503756e-03, -7.83264730e-03,  3.19121126e-03, -1.50027629e-02,
#          4.75921319e-04,  5.00955619e-03, -1.25764310e-03, -5.44507243e-03,
#         -1.57925510e-03, -2.19089235e-03,  1.62292621e-03,  8.46134499e-03,
#          3.43065732e-03,  3.93456127e-03,  3.52358492e-03, -1.23998243e-03,
#          1.36766024e-02,  2.08690413e-03, -5.54620381e-03, -2.27881223e-03,

# 인접행렬 만드는 작업
## cosin metrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim = cosine_similarity(doc_embedding_list, doc_embedding_list)
type(cosine_sim), cosine_sim.shape

(numpy.ndarray, (2381, 2381))

In [ ]:
cosine_sim[3]

array([0.76517904, 0.7289075 , 0.7382629 , ..., 0.7205972 , 0.6711928 ,
       0.75324404], dtype=float32)

# predict

In [ ]:
title = 'The Da Vinci Code'
title = 'The Four Pillars of Investing' # 3

In [ ]:
idx = 3
# numpy 형태의 cosine_sim
sim_scores = list(enumerate(cosine_sim[3]))
print(sim_scores)
# rawdata에서 index=3의 행과 유사도를 비교한 것이므로 (3,~)은 1에 가까운 수치

[(0, 0.76517904), (1, 0.7289075), (2, 0.7382629), (3, 0.99999964), (4, 0.7084964), (5, 0.7805427), (6, 0.8593215), (7, 0.7965765), (8, 0.73943883), (9, 0.7254228), (10, 0.7626609), (11, 0.7512931), (12, 0.72597575), (13, 0.7643095), (14, 0.6824963), (15, 0.75938034), (16, 0.7780046), (17, 0.76274765), (18, 0.74791276), (19, 0.72259754), (20, 0.7390192), (21, 0.736445), (22, 0.71036184), (23, 0.7436199), (24, 0.79294527), (25, 0.64615715), (26, 0.7470798), (27, 0.7641254), (28, 0.72671926), (29, 0.758363), (30, 0.7627596), (31, 0.74568564), (32, 0.7026745), (33, 0.73097885), (34, 0.7818177), (35, 0.756035), (36, 0.76585084), (37, 0.7577896), (38, 0.76890063), (39, 0.751515), (40, 0.7727772), (41, 0.75017476), (42, 0.7340061), (43, 0.78257483), (44, 0.7181958), (45, 0.7281888), (46, 0.74544686), (47, 0.7877703), (48, 0.76137036), (49, 0.66511476), (50, 0.70324343), (51, 0.6985787), (52, 0.7528975), (53, 0.72967654), (54, 0.77925795), (55, 0.7351316), (56, 0.7724032), (57, 0.70177525), (5

In [ ]:
sim_scores_list = sorted(sim_scores, key = lambda x:x[1], reverse=True)

In [ ]:
sim_scores_list[1:11]

[(6, 0.8593215),
 (225, 0.85243374),
 (537, 0.84788424),
 (606, 0.84673786),
 (331, 0.84571534),
 (160, 0.84255934),
 (1083, 0.84130746),
 (887, 0.8391184),
 (358, 0.8390085),
 (907, 0.8389222)]

In [ ]:
topsim10 = sim_scores_list[1:11]
topsim10

[(6, 0.8593215),
 (225, 0.85243374),
 (537, 0.84788424),
 (606, 0.84673786),
 (331, 0.84571534),
 (160, 0.84255934),
 (1083, 0.84130746),
 (887, 0.8391184),
 (358, 0.8390085),
 (907, 0.8389222)]

# 최종 유사 title 10순위 출력

In [ ]:
for i in range(len(topsim10)):
  print(df_data.iloc[topsim10[i][0]]['title'])
  # print(topsim10[i][0])

The Man Who Solved the Market: How Jim Simons Launched the Quant Revolution
Disrupt You!: Master Personal Transformation, Seize Opportunity, and Thrive in the Era of Endless Innovation
The Future of Capitalism: Facing the New Anxieties
Principles: Life and Work
Case Interview Secrets: A Former McKinsey Interviewer Reveals How to Get Multiple Job Offers in Consulting
The Startup Playbook: Secrets of the Fastest-Growing Startups from Their Founding Entrepreneurs
Go Put Your Strengths to Work: 6 Powerful Steps to Achieve Outstanding Performance
Creative Confidence: Unleashing the Creative Potential Within Us All
Knowing Your Value: Women, Money, and Getting What You're Worth
Orbiting the Giant Hairball: A Corporate Fool's Guide to Surviving with Grace
